In [1]:
from pathlib import Path
import json
import pandas as pd
from inverse_modelling_tfo.data_pipelines.fetal_conc_groups import dan_iccps_pencil1, generate_grouping_from_config
from inverse_modelling_tfo.data import config_based_normalization
from inverse_modelling_tfo.data.intensity_interpolation import (
    interpolate_exp,
    get_interpolate_fit_params,
    exp_piecewise_affine,
)
from inverse_modelling_tfo.data.interpolation_function_zoo import *
from inverse_modelling_tfo.features.build_features import (
    FetalACFeatureBuilder,
    RowCombinationFeatureBuilder,
    TwoColumnOperationFeatureBuilder,
    FetalACbyDCFeatureBuilder,
    LogTransformFeatureBuilder,
    ConcatenateFeatureBuilder,
)
from inverse_modelling_tfo.features.data_transformations import (
    LongToWideIntensityTransformation,
    ToFittingParameterTransformation,
)

# Data Setup
# ==========================================================================================
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "I1_and_I2.pkl"
out_dest = out_dest = Path(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/processed_data') / "pulsation_ratio_interp.pkl"
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "logI2_by_I1.pkl"
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "processed1_max_long_range.pkl"
config_dest = out_dest.with_suffix(".json")

in_src = Path(r"/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/pencil2.pkl")
config_src = in_src.with_suffix(".json")

fconc_rounding = 2
grouping_map = generate_grouping_from_config(config_src, fconc_rounding)


data = pd.read_pickle(in_src)
config_based_normalization(data, config_src)

# ==========================================================================================
data = data.drop(columns="Uterus Thickness")

In [2]:
import tqdm
# Data Processing

accumulated_data = pd.DataFrame()
all_depths = data["Maternal Wall Thickness"].unique()
all_depths.sort()

for chosen_depth in tqdm.tqdm(all_depths):
    temp_data = data[data["Maternal Wall Thickness"] == chosen_depth].copy()
    # Interpolate intensity to remove noise
    temp_data = interpolate_exp(temp_data, weights=(1, 0.6), interpolation_function=exp_piecewise_affine, break_indices=[4, 12, 20])
    temp_data["Intensity"] = temp_data["Interpolated Intensity"]  # Replace OG intensity with interpolated intensity
    temp_data = temp_data.drop(columns="Interpolated Intensity")  # Cleanup
    accumulated_data = pd.concat([accumulated_data, temp_data], ignore_index=True)

data = accumulated_data

100%|██████████| 31/31 [20:52<00:00, 40.40s/it]


In [3]:
data.head()

,Wave Int,SDD,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,Intensity
0,2.0,10,4.0,11.0,0.9,11.0,0.2,1.019389e-05
1,2.0,15,4.0,11.0,0.9,11.0,0.2,5.539847e-07
2,2.0,19,4.0,11.0,0.9,11.0,0.2,5.951803e-08
3,2.0,24,4.0,11.0,0.9,11.0,0.2,5.446266e-09
4,2.0,28,4.0,11.0,0.9,11.0,0.2,7.979386e-10


In [4]:
out_dest = out_dest = Path(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/processed_data') / "pulsation_ratio_interp.pkl"
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "logI2_by_I1.pkl"
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "processed1_max_long_range.pkl"
config_dest = out_dest.with_suffix(".json")

# Define data transformers
data_transformer = LongToWideIntensityTransformation()
# fitting_param_transformer = ToFittingParameterTransformation()

# Transform data
# fitting_params = fitting_param_transformer.transform(data)
data = data_transformer.transform(data)
labels = data_transformer.get_label_names()
intensity_columns = data_transformer.get_feature_names()

# Cleanup
data.dropna(inplace=True)

# Create fetal conc. grouping column - used for generating the AC component/which rows to choose for pairing
data["FconcCenters"] = data["Fetal Hb Concentration"].round(fconc_rounding).map(grouping_map)
labels = labels + ["FconcCenters"]  # This new column grouping should also be treated as a label
# fitting_params['FconcCenters'] = data['FconcCenters']
fixed_columns = [
    "Maternal Wall Thickness",
    "Maternal Hb Concentration",
    "Maternal Saturation",
    "Fetal Saturation",
    "FconcCenters",
]

fb1 = FetalACbyDCFeatureBuilder("FconcCenters", "comb", intensity_columns, labels, "max")

# Build features
data = fb1(data)


# Create Config file
# ==========================================================================================
# NOT AUTOGENRATED! MUST BE DONE MANUALLY FOR EACH PIPELINE
config = {
    "labels": fb1.get_label_names(),
    "features": fb1.get_feature_names(),
    "feature_builder_txt": str(fb1),
    "preprocessing_description": "Interpolation -> Detector Normalization -> Long to Wide -> Row Combination -> PR",
    "comments": "Using Linear Exponential Interpolation to calculate I1 and I2",
    "data used": "/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/pencil2.pkl",
}

# Save data and config
# ==========================================================================================
data.to_pickle(out_dest)

with open(config_dest, "w+", encoding="utf-8") as outfile:
    json.dump(config, outfile)